In [110]:
import mlflow
import pandas as pd
import numpy as np
import argparse
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.datasets.mnist import load_data
import os
import datetime
import uuid
import time
import logging
from hyperopt import (fmin, atpe, hp, STATUS_OK, STATUS_FAIL, Trials)
idx = pd.IndexSlice

# Data process

In [111]:
# INPUTS
DATA_PATH = '/Users/camilovelasquez/Desktop/Documents/Datasets/WISDM-Smartphones/wisdm-dataset/raw'
ids = np.arange(1600, 1650)
devices = ['phone']
sensors = ['accel']
activities = ['A', 'B']
time_taken = 3000
time_split = 100

In [112]:
train_batch_size = 8
eval_batch_size = 16
epochs = 5

train_size = 2500
valid_size = 220
test_size = 220

In [113]:
# Preprocessing data
def read_WISDM_data(DATA_PATH, ids=np.arange(1600, 1650), 
                    devices=['phone'], sensors=['accel']):
    """Read from DATA PATH and create a pandas table from it"""
    i = 0
    for current_id in ids:
        for current_device in devices:
            for current_sensor in sensors:
                file_path = os.path.join(DATA_PATH, current_device, current_sensor, 
                                         'data_{}_{}_{}.txt'.format(current_id, current_sensor, current_device))
                if i==0:
                    table = pd.read_csv(file_path, delimiter=',', 
                                        names=['ID', 'Activity Label', 'Timestamp', 'x', 'y', 'z'], 
                                        lineterminator='\n')
                else:
                    aux = pd.read_csv(file_path, delimiter=',', 
                                      names=['ID', 'Activity Label', 'Timestamp', 'x', 'y', 'z'], 
                                        lineterminator='\n')
                    table = pd.concat([table, aux], axis=0)
                i+=1
    table.loc[:,'z'] = table.z.str.replace(';','').astype(np.float32)
    return table

def transform_data(table, time_taken, time_split):
    """Transform data from raw table into a zip of (features, labels),
        where features has shape (samples, time_steps, features), and labels (samples,)"""
    table = table.set_index(['ID', 'Activity Label'])
    table = table.groupby(['ID', 'Activity Label']).head(time_taken)
    timestamp_edit = np.tile(np.arange(0,time_split), int(table.shape[0]/time_split))
    table['Timestamp'] = timestamp_edit
    table = table.reset_index().set_index(['ID', 'Activity Label', 'Timestamp'], append=True)
    features = table.values.reshape((int(table.shape[0]/time_split), time_split, table.shape[1]))
    labels = table.reset_index()['Activity Label']\
        .values[np.arange(0,int(table.shape[0]/time_split)*time_split, time_split)]
    return features, labels

def preprocessing_data(table, time_taken=3000, time_split=100, activities=['A', 'B']):
    """Preprocess table and convert it into tf dataset"""
    features, labels = transform_data(table, time_taken=time_taken, time_split=time_split)
    dataset = tf.data.Dataset.from_tensor_slices(({'feature': features}, {'label': labels}))
    ds = dataset.filter(lambda x, y: tf.reduce_any(tf.equal(y['label'], activities))==True)
    ds = ds.map(label2prob)
    return ds

def label2prob(feature, label):
    new_label = tf.where(tf.equal(label['label'], 'A'), 1, 0)
    label['label'] = new_label
    return feature, label

In [114]:
table = read_WISDM_data(DATA_PATH, ids=ids, devices=devices, sensors=sensors)
ds = preprocessing_data(table, time_taken=time_taken, time_split=time_split, activities=activities)

# Adding params

In [115]:
params = {'sampling': hp.choice('sampling', [
                                {'type' : 'under', 
                                 'low_perc' : hp.choice('low_perc', [0.1, 0.2, 0.3])},
                                {'type' : 'over', 
                                 'low_perc' : hp.choice('low_perc', [0.1, 0.2, 0.3])}]),
          'batch_size' : hp.choice('batch_size', [64, 128, 256]),
          'test_mode' : True,
          'weighted' : hp.choice('weighted', [None, 'regular', 'double']),
          'eval_batch_size' : 256,
          'optimizer': hp.choice('optimizer', [
              {'type': 'adam', 
               'lr' : hp.loguniform('lr_mult', -1, 3)}, 
              {'type' : 'SGD', 
               'lr' :  hp.loguniform('lr_mult', -1, 3)} ]),
          'loss': hp.choice('loss', [
              {'type': 'binary_crossentropy'}, 
              {'type' : 'focal_loss', 
               'alpha' :  hp.uniform('alpha', -1, 3), 
               'gamma' : hp.choice('gamma', [0.5, 0.75, 1.0])} ]),
         }

In [116]:
params = {'batch_size' : hp.choice('batch_size', [8, 16, 32]), 
          'weighted' : hp.choice('weighted', [True, False])}

In [117]:
EXPERIMENT_NAME = 'testing_with_hyper_opt_3'
#mlflow.set_tracking_uri(URI) # If you have an instance
mlflow.set_experiment(EXPERIMENT_NAME)
print("Experiment Name: ", EXPERIMENT_NAME)

INFO: 'testing_with_hyper_opt_3' does not exist. Creating a new experiment
Experiment Name:  testing_with_hyper_opt_3


# Building params

In [118]:
#Model params
def build_model(time_split):
    inputs = tf.keras.Input(shape=(time_split, 3), name='feature')
    x = tf.keras.layers.BatchNormalization(axis=2)(inputs)
    x_1 = tf.keras.layers.Conv1D(filters=32, kernel_size=15)(x)
    x_1 = tf.keras.layers.GlobalMaxPool1D()(x_1)
    x_3 = tf.keras.layers.Conv1D(filters=32, kernel_size=31)(x)
    x_3 = tf.keras.layers.GlobalMaxPool1D()(x_3)
    x_5 = tf.keras.layers.Conv1D(filters=32, kernel_size=63)(x)
    x_5 = tf.keras.layers.GlobalMaxPool1D()(x_5)
    x = tf.keras.layers.Concatenate()([x_1, x_3, x_5])
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='label')(x)
    model = tf.keras.Model(inputs=[inputs], outputs=output)
    return model

def build_optimizer():
    optimizer='adam'
    return optimizer

def build_loss():
    loss='binary_crossentropy'
    return loss

def build_metrics():
    metrics=['accuracy']
    return metrics

def compile_model(model, optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']):
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

def train_model(model, train_ds=None, valid_ds=None):
    model.fit()

def myprint(s, filename='modelsummary.txt'):
    with open(filename,'w+') as f:
        print(s, file=f)
    
def build_class_weights(weight_type):
    if weight_type:
        class_weight = {1: 0.8, 0: 0.2}
    else:
        class_weight = None
    return class_weight

In [119]:
def build_callbacks(params):
    callbacks = []
    if 'es' in params['train']['params']['callbacks']:
        es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)
        callbacks.append(es_callback)
    elif 'cp' in params['train']['params']['callbacks']:
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                         save_weights_only=True,
                                                         verbose=1)
        callbacks.append(cp_callback)
    return callbacks

In [120]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)
callbacks = [es_callback]

In [121]:
def transform_key_value_pair(kv_pairs, key, dictionary):
    if type(dictionary) is dict:
        for new_key, new_value in dictionary.items():
            if key is not None:
                transform_key_value_pair(kv_pairs, key + '_' + str(new_key), new_value)
            else:
                transform_key_value_pair(kv_pairs, new_key, new_value)    
    else:
        kv_pairs[key] = dictionary
        
def example(**args):
    print(args)
    for arg in args:
        print(arg)

In [122]:
def objetive(params):
    summary_filename = 'summary.txt'
    run_name = 'model_' + str(uuid.uuid4())[:5]
    with mlflow.start_run(run_name=run_name) as run:
        #Model
        model = build_model(time_split=time_split)
        optimizer = build_optimizer()
        loss = build_loss()
        metrics = build_metrics()
        model = compile_model(model=model, optimizer=optimizer, loss=loss, metrics=metrics)
        model.summary(print_fn=lambda x: myprint(x, summary_filename))

        #Build dataset
        ds_train = ds.take(train_size)
        ds_train = ds_train.repeat(count=epochs)
        ds_train = ds_train.batch(params['batch_size'])
        ds_train = ds_train.prefetch(1)

        ds_valid = ds.skip(train_size).take(valid_size)
        ds_valid = ds_valid.repeat(count=1)
        ds_valid = ds_valid.batch(params['batch_size'])
        ds_valid = ds_valid.prefetch(1)

        ds_test = ds.skip(train_size+valid_size)
        ds_test = ds_test.batch(eval_batch_size)
        
        #Train
        
        history = model.fit(ds_train, 
                    validation_data=ds_valid,
                    steps_per_epoch=train_size//params['batch_size'],  
                    epochs=epochs, 
                    callbacks=callbacks,
                    verbose=1, 
                    class_weight=build_class_weights(params['weighted']),
                    shuffle=False)
        model.save('model_'+ run_name + '.h5')
        
        # Metrics
        mlflow.log_metric('val_accuracy', max(history.history['val_accuracy']))
        mlflow.log_param('batch_size', params['batch_size'])
        mlflow.log_param('weighted', params['weighted'])
        # Artifacts
        mlflow.log_artifact(summary_filename)
        mlflow.log_artifact('model_'+ run_name + '.h5')
        
        mlflow.end_run()
    return {'loss': -max(history.history['val_accuracy']), 'status': STATUS_OK}

In [123]:
trials = Trials()
_ = fmin(objetive, params, trials=trials, algo=tpe.suggest, max_evals=50)

Train for 156 steps                                   
Epoch 1/5                                             
  1/156 [..............................]              
 - ETA: 2:01 - loss: 0.9884 - accuracy: 0.0000e+00    
                                                     
  7/156 [>.............................]              
 - ETA: 17s - loss: 0.7093 - accuracy: 0.5714         
                                                      
 12/156 [=>............................]              
 - ETA: 10s - loss: 0.6863 - accuracy: 0.5729         
                                                      
 17/156 [==>...........................]              
 - ETA: 7s - loss: 0.6426 - accuracy: 0.6176          
                                                     
 22/156 [===>..........................]              
 - ETA: 6s - loss: 0.6432 - accuracy: 0.6193          
                                                     
 27/156 [====>.........................]              
 - ETA: 5s - 

13/14 [==========================>...]                
 - ETA: 0s - loss: 0.0168 - accuracy: 1.0000          
                                                     
156/14 [==============================================================================================================================================================================================================================================================================================================================================]
 - 2s 15ms/step - loss: 0.0393 - accuracy: 0.9952 - val_loss: 0.1114 - val_accuracy: 0.9864

Epoch 5/5                                             
 1/14 [=>............................]                
 - ETA: 0s - loss: 0.0490 - accuracy: 1.0000          
                                                     
 8/14 [================>.............]                
 - ETA: 0s - loss: 0.0108 - accuracy: 1.0000          
                                                     
13/14 [=========

77/78 [============================>.]                                           
 - ETA: 0s - loss: 0.6095 - accuracy: 0.6420                                     
                                                                                
78/78 [==============================]                                           
 - 3s 44ms/step - loss: 0.6100 - accuracy: 0.6390 - val_loss: 0.4005 - val_accuracy: 0.7318

Epoch 2/5                                                                        
1/7 [===>..........................]                                             
 - ETA: 0s - loss: 0.3241 - accuracy: 1.0000                                     
                                                                                
4/7 [================>.............]                                             
 - ETA: 0s - loss: 0.3416 - accuracy: 0.8828                                     
                                                                                
78/7 [==

 - ETA: 2s - loss: 0.6347 - accuracy: 0.6732                                     
                                                                                
117/312 [==========>...................]                                         
 - ETA: 2s - loss: 0.6290 - accuracy: 0.6859                                     
                                                                                
121/312 [==========>...................]                                         
 - ETA: 2s - loss: 0.6343 - accuracy: 0.6746                                     
                                                                                
128/312 [===========>..................]                                         
 - ETA: 2s - loss: 0.6220 - accuracy: 0.6924                                     
                                                                                
138/312 [============>.................]                                         
 - ETA: 2s - loss: 0

 - 2s 8ms/step - loss: 0.2112 - accuracy: 0.9543 - val_loss: 0.2677 - val_accuracy: 0.9045

Epoch 3/5                                                                        
 1/28 [>.............................]                                           
 - ETA: 0s - loss: 0.1714 - accuracy: 1.0000                                     
                                                                                
10/28 [=========>....................]                                           
 - ETA: 0s - loss: 0.0312 - accuracy: 1.0000                                     
                                                                                
21/28 [=====================>........]                                           
 - ETA: 0s - loss: 0.0290 - accuracy: 1.0000                                     
                                                                                
312/28 [===================================================================================

...

 - 2s 14ms/step - loss: 0.1273 - accuracy: 0.9816 - val_loss: 0.2193 - val_accuracy: 0.8727

Epoch 4/5                                                                         
 1/14 [=>............................]                                            
 - ETA: 0s - loss: 0.0720 - accuracy: 1.0000                                      
                                                                                 
 6/14 [===========>..................]                                            
 - ETA: 0s - loss: 0.0273 - accuracy: 1.0000                                      
                                                                                 
12/14 [========================>.....]                                            
 - ETA: 0s - loss: 0.0292 - accuracy: 1.0000                                      
                                                                                 
156/14 [========================================================================

                                                                                 
78/78 [==============================]                                            
 - 4s 48ms/step - loss: 0.6055 - accuracy: 0.6567 - val_loss: 0.4252 - val_accuracy: 0.7091

Epoch 2/5                                                                         
1/7 [===>..........................]                                              
 - ETA: 0s - loss: 0.3188 - accuracy: 1.0000                                      
                                                                                 
4/7 [================>.............]                                              
 - ETA: 0s - loss: 0.3599 - accuracy: 0.9062                                      
                                                                                 
78/7 [=============================================================================================================================================================

                                                                                 
 86/156 [===============>..............]                                          
 - ETA: 1s - loss: 0.6012 - accuracy: 0.6948                                      
                                                                                 
 91/156 [================>.............]                                          
 - ETA: 1s - loss: 0.5964 - accuracy: 0.6937                                      
                                                                                 
 97/156 [=================>............]                                          
 - ETA: 1s - loss: 0.6027 - accuracy: 0.6785                                      
                                                                                 
102/156 [==================>...........]                                          
 - ETA: 0s - loss: 0.5943 - accuracy: 0.6869                                      
        

 - 2s 14ms/step - loss: 0.0099 - accuracy: 0.9984 - val_loss: 0.0976 - val_accuracy: 0.9773

Train for 312 steps                                                               
Epoch 1/5                                                                         
  1/312 [..............................]                                          
 - ETA: 3:51 - loss: 2.4524 - accuracy: 0.0000e+00                                
                                                                                 
  8/312 [..............................]                                          
 - ETA: 30s - loss: 1.0178 - accuracy: 0.4688                                     
                                                                                  
 16/312 [>.............................]                                          
 - ETA: 16s - loss: 0.9272 - accuracy: 0.4688                                     
                                                                              

                                                                                 
263/312 [========================>.....]                                          
 - ETA: 0s - loss: 0.5547 - accuracy: 0.7414                                      
                                                                                 
271/312 [=========================>....]                                          
 - ETA: 0s - loss: 0.5452 - accuracy: 0.7486                                      
                                                                                 
278/312 [=========================>....]                                          
 - ETA: 0s - loss: 0.5376 - accuracy: 0.7540                                      
                                                                                 
286/312 [==========================>...]                                          
 - ETA: 0s - loss: 0.5332 - accuracy: 0.7583                                      
        